In [1]:
# Clone the repository
!git clone https://github.com/FERMat-ML/OMatG.git
%cd OMatG

# Install the package and dependencies
!pip install -e .

Cloning into 'OMatG'...
remote: Enumerating objects: 4763, done.
remote: Total 4763 (delta 0), reused 0 (delta 0), pack-reused 4763 (from 1)
Receiving objects: 100% (4763/4763), 174.13 MiB | 30.92 MiB/s, done.
Resolving deltas: 100% (3244/3244), done.
Updating files: 100% (72/72), done.
/content/OMatG
Obtaining file:///content/OMatG
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.0/108.0 kB 7.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.6/55.6 kB 3.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━

In [1]:
!pip install freud-analysis

  Using cached freud_analysis-3.5.0-cp312-abi3-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl.metadata (6.2 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 25.8 MB/s eta 0:00:00


In [3]:
from google.colab import drive
drive.mount('/content/drive')
import sys
import os
import torch

# Need to edit the file_path to where Voronoi.py is saved
# file_path_in_drive = 'My Drive/Colab Notebooks' # ethan's
file_path_in_drive = 'My Drive/martiniani_lab_research' # kosta's
full_path = os.path.join('/content/drive', file_path_in_drive)
sys.path.append(full_path)

from Voronoi import VoronoiPhantomCellGenerator
import freud
import numpy as np
print("Successfully imported Voronoi!")

Mounted at /content/drive
Successfully imported Voronoi!


In [4]:
# create Voronoi Generator
VoronoiGenerator = VoronoiPhantomCellGenerator(
  desired_atom_count=20,
  dist_eval="min"
)

In [5]:
from omg.datamodule import DataModule

# Create DataModule instances and load from LMDB
train_data = DataModule()
train_data.add_from_lmdb("OMatG/omg/data/mp_20/train.lmdb")

val_data = DataModule()
val_data.add_from_lmdb("OMatG/omg/data/mp_20/val.lmdb")

print(f"\nTraining set size: {len(train_data)}")
print(f"Validation set size: {len(val_data)}")


2025-10-31 01:45:22.394 | INFO     | omg.datamodule.datamodule:add_from_lmdb:876 - Creating new LMDB file: dataset_b5db469d6374119b3d4fe2c93e0a0fda.lmdb


Adding 27136 configurations to LMDB from OMatG/omg/data/mp_20/train.lmdb


100%|██████████| 27136/27136 [00:00<00:00, 112151.85it/s]
2025-10-31 01:45:23.368 | INFO     | omg.datamodule.datamodule:add_from_lmdb:876 - Creating new LMDB file: dataset_5ecb29936171c877a54bf448e32a93cc.lmdb


Adding 9047 configurations to LMDB from OMatG/omg/data/mp_20/val.lmdb


100%|██████████| 9047/9047 [00:00<00:00, 147622.87it/s]


Training set size: 27136
Validation set size: 9047


In [6]:
from omg.datamodule.dataloader import OMGTorchDataset, OMGDataModule
from torch_geometric.loader import DataLoader

# Assuming you have train_data and val_data as DataModule instances
# Convert to PyTorch Geometric datasets
train_dataset = OMGTorchDataset(
    dataset=train_data,           # Your DataModule instance
    convert_to_fractional=False,
    niggli=False                  # Apply Niggli cell reduction (optional)
)

val_dataset = OMGTorchDataset(
    dataset=val_data,
    convert_to_fractional=False,
    niggli=False
)

print(f"Train dataset size: {len(train_dataset)}")
print(f"Train dataset type: {type(train_dataset)}")
print(f"Val dataset size: {len(val_dataset)}")


Train dataset size: 27136
Train dataset type: <class 'omg.datamodule.dataloader.OMGTorchDataset'>
Val dataset size: 9047


#### Data Example

In [7]:
single_data = train_dataset[4]
print(single_data)
cell = single_data.cell[0].numpy()
x_vec, y_vec, z_vec = cell[0], cell[1], cell[2]

# Assuming that positions are in cartesian coords not fractional coords
positions = single_data.pos.numpy()
new_points = positions.copy()

# Calculate iterations once
iterations = 20 - single_data.n_atoms.item()

# Generate ghost atoms
for _ in range(iterations):
    next_point = VoronoiGenerator._get_next_point(new_points, x_vec, y_vec, z_vec)
    new_points = np.vstack([new_points, next_point])


OMGData(
  n_atoms=16,
  batch=[16],
  species=[16],
  cell=[1, 3, 3],
  pos=[16, 3],
  property={}
)


#### Data Example Plot

In [8]:
import plotly.graph_objects as go
import pandas as pd

# Create DataFrames for both
df_vertices = pd.DataFrame(new_points, columns=['x', 'y', 'z'])
df_vertices['type'] = 'New Points'

df_points = pd.DataFrame(positions, columns=['x', 'y', 'z'])
df_points['type'] = 'Original Points'

# Create figure
fig = go.Figure()

# Add scatter points
fig.add_trace(go.Scatter3d(
    x=df_vertices['x'], y=df_vertices['y'], z=df_vertices['z'],
    mode='markers',
    name='Next Point',
    marker=dict(size=5, opacity=0.6)
))

fig.add_trace(go.Scatter3d(
    x=df_points['x'], y=df_points['y'], z=df_points['z'],
    mode='markers',
    name='Original Points',
    marker=dict(size=5, opacity=0.6)
))

# Define the 8 vertices of the parallelepiped
# Starting from origin
origin = np.array([0, 0, 0])
vertices = [
    origin,                          # 0: (0,0,0)
    x_vec,                          # 1: x_vec
    y_vec,                          # 2: y_vec
    z_vec,                          # 3: z_vec
    x_vec + y_vec,                  # 4: x_vec + y_vec
    x_vec + z_vec,                  # 5: x_vec + z_vec
    y_vec + z_vec,                  # 6: y_vec + z_vec
    x_vec + y_vec + z_vec          # 7: x_vec + y_vec + z_vec
]

# Define the 12 edges (pairs of vertex indices to connect)
edges = [
    (0, 1), (0, 2), (0, 3),  # edges from origin
    (1, 4), (1, 5),          # edges from x_vec
    (2, 4), (2, 6),          # edges from y_vec
    (3, 5), (3, 6),          # edges from z_vec
    (4, 7), (5, 7), (6, 7)   # edges to opposite corner
]

# Draw each edge
for i, j in edges:
    v1, v2 = vertices[i], vertices[j]
    fig.add_trace(go.Scatter3d(
        x=[v1[0], v2[0]],
        y=[v1[1], v2[1]],
        z=[v1[2], v2[2]],
        mode='lines',
        line=dict(color='black', width=3),
        showlegend=False,
        hoverinfo='none'
    ))

fig.update_layout(
    title='Parallelepiped with Points',
    width=500,  # Width in pixels
    height=500,  # Height in pixels
    autosize=False,  # Disable autosize to use fixed dimensions
    scene=dict(
        xaxis_title='X',
        yaxis_title='Y',
        zaxis_title='Z'
    )
)

fig.show()


## New Dataset Generation

In [9]:
new_data_list = []

for idx in range(10):

  single_data = train_dataset[idx]
  cell = single_data.cell[0].numpy()
  x_vec, y_vec, z_vec = cell[0], cell[1], cell[2]

  # Assuming that positions are in cartesian coords not fractional coords
  positions = single_data.pos.numpy()
  new_points = positions.copy()

  # Calculate iterations once
  iterations = 20 - single_data.n_atoms.item()

  # Generate ghost atoms
  for _ in range(iterations):
      next_point = VoronoiGenerator._get_next_point(new_points, x_vec, y_vec, z_vec)
      new_points = np.vstack([new_points, next_point])

  # Update pos, n_atoms, batch
  single_data.pos = torch.from_numpy(new_points).to(dtype=torch.float64)
  single_data.n_atoms = torch.tensor(20, dtype=torch.long)
  single_data.batch = torch.zeros(20, dtype=torch.long)  # All atoms belong to config 0

  # Update species efficiently
  num_ghosts = iterations
  ghost_species = torch.full((num_ghosts,), -1, dtype=torch.long)
  single_data.species = torch.cat([single_data.species.long(), ghost_species])

  new_data_list.append(single_data)

  if idx == 1:
    # --- Visualization Code ---

    # Create DataFrames for both
    df_points = pd.DataFrame(positions, columns=['x', 'y', 'z'])
    df_points['type'] = 'Original Points'

    # Assuming the last 'iterations' points in new_points are the ghost atoms
    ghost_points = new_points[-iterations:]
    df_ghost_points = pd.DataFrame(ghost_points, columns=['x', 'y', 'z'])


    # Create figure
    fig = go.Figure()

    # Add scatter points for original points
    fig.add_trace(go.Scatter3d(
        x=df_points['x'], y=df_points['y'], z=df_points['z'],
        mode='markers',
        name='Original Points',
        marker=dict(size=5, opacity=0.6, color='blue')
    ))

    # Add scatter points for new (ghost) points
    fig.add_trace(go.Scatter3d(
        x=df_ghost_points['x'], y=df_ghost_points['y'], z=df_ghost_points['z'],
        mode='markers',
        name='Ghost Points',
        marker=dict(size=5, opacity=0.6, color='red')
    ))


    # Define the 8 vertices of the parallelepiped
    # Starting from origin
    origin = np.array([0, 0, 0])
    vertices = [
        origin,                          # 0: (0,0,0)
        x_vec,                          # 1: x_vec
        y_vec,                          # 2: y_vec
        z_vec,                          # 3: z_vec
        x_vec + y_vec,                  # 4: x_vec + y_vec
        x_vec + z_vec,                  # 5: x_vec + z_vec
        y_vec + z_vec,                  # 6: y_vec + z_vec
        x_vec + y_vec + z_vec          # 7: x_vec + y_vec + z_vec
    ]

    # Define the 12 edges (pairs of vertex indices to connect)
    edges = [
        (0, 1), (0, 2), (0, 3),  # edges from origin
        (1, 4), (1, 5),          # edges from x_vec
        (2, 4), (2, 6),          # edges from y_vec
        (3, 5), (3, 6),          # edges from z_vec
        (4, 7), (5, 7), (6, 7)   # edges to opposite corner
    ]

    # Draw each edge
    for i, j in edges:
        v1, v2 = vertices[i], vertices[j]
        fig.add_trace(go.Scatter3d(
            x=[v1[0], v2[0]],
            y=[v1[1], v2[1]],
            z=[v1[2], v2[2]],
            mode='lines',
            line=dict(color='black', width=3),
            showlegend=False,
            hoverinfo='none'
        ))

    fig.update_layout(
        title=f'Sample {idx}: Parallelepiped with Points',
        width=500,  # Width in pixels
        height=500,  # Height in pixels
        autosize=False,  # Disable autosize to use fixed dimensions
        scene=dict(
            xaxis_title='X',
            yaxis_title='Y',
            zaxis_title='Z'
        )
    )

    fig.show()
    # --- End Visualization Code ---

In [10]:
from torch_geometric.data import InMemoryDataset
import torch
import os

# Create the directory if it doesn't exist
os.makedirs('processed', exist_ok=True)

# Save as processed data
torch.save(new_data_list, 'processed/modified_data.pt')

In [11]:
import torch

# Load the dataset
new_data_list = torch.load('processed/modified_data.pt', weights_only=False)

# Check what you loaded
print(f"Loaded {len(new_data_list)} samples")
print(f"First sample: {new_data_list[0]}")

Loaded 10 samples
First sample: OMGData(
  n_atoms=20,
  batch=[20],
  species=[20],
  cell=[1, 3, 3],
  pos=[20, 3],
  property={}
)


In [14]:
def plot_crystal_with_points(single_data, idx):
  """
  Plots a crystal structure with original and ghost points.

  Args:
      single_data: A data object containing cell, positions, and species.
      idx: The index of the sample being plotted.
  """
  cell = single_data.cell[0].numpy()
  x_vec, y_vec, z_vec = cell[0], cell[1], cell[2]

  # Assuming that positions are in cartesian coords not fractional coords
  positions = single_data.pos.numpy()
  species = single_data.species.numpy()

  # Create DataFrames for both original and ghost points
  df_points = pd.DataFrame(positions, columns=['x', 'y', 'z'])
  df_points['species'] = species
  df_points['type'] = df_points['species'].apply(lambda s: 'Original Points' if s != -1 else 'Ghost Points')

  # Create figure
  fig = go.Figure()

  # Add scatter points for original points
  fig.add_trace(go.Scatter3d(
      x=df_points[df_points['type'] == 'Original Points']['x'],
      y=df_points[df_points['type'] == 'Original Points']['y'],
      z=df_points[df_points['type'] == 'Original Points']['z'],
      mode='markers',
      name='Original Points',
      marker=dict(size=5, opacity=0.6, color='blue')
  ))

  # Add scatter points for new (ghost) points
  fig.add_trace(go.Scatter3d(
      x=df_points[df_points['type'] == 'Ghost Points']['x'],
      y=df_points[df_points['type'] == 'Ghost Points']['y'],
      z=df_points[df_points['type'] == 'Ghost Points']['z'],
      mode='markers',
      name='Ghost Points',
      marker=dict(size=5, opacity=0.6, color='red')
  ))

  # Define the 8 vertices of the parallelepiped
  origin = np.array([0, 0, 0])
  vertices = [
      origin,                          # 0: (0,0,0)
      x_vec,                          # 1: x_vec
      y_vec,                          # 2: y_vec
      z_vec,                          # 3: z_vec
      x_vec + y_vec,                  # 4: x_vec + y_vec
      x_vec + z_vec,                  # 5: x_vec + z_vec
      y_vec + z_vec,                  # 6: y_vec + z_vec
      x_vec + y_vec + z_vec          # 7: x_vec + y_vec + z_vec
  ]

  # Define the 12 edges (pairs of vertex indices to connect)
  edges = [
      (0, 1), (0, 2), (0, 3),  # edges from origin
      (1, 4), (1, 5),          # edges from x_vec
      (2, 4), (2, 6),          # edges from y_vec
      (3, 5), (3, 6),          # edges from z_vec
      (4, 7), (5, 7), (6, 7)   # edges to opposite corner
  ]

  # Draw each edge
  for i, j in edges:
      v1, v2 = vertices[i], vertices[j]
      fig.add_trace(go.Scatter3d(
          x=[v1[0], v2[0]],
          y=[v1[1], v2[1]],
          z=[v1[2], v2[2]],
          mode='lines',
          line=dict(color='black', width=3),
          showlegend=False,
          hoverinfo='none'
      ))

  fig.update_layout(
      title=f'Sample {idx}: Newly Loaded Data',
      width=500,  # Width in pixels
      height=500,  # Height in pixels
      autosize=False,  # Disable autosize to use fixed dimensions
      scene=dict(
          xaxis_title='X',
          yaxis_title='Y',
          zaxis_title='Z'
      )
  )

  fig.show()

# Iterate through all samples in the loaded data and plot
for idx, single_data in enumerate(new_data_list):
  plot_crystal_with_points(single_data, idx)

# Task
Save the modified data, which is currently in a list of `Data` objects, back into LMDB format, maintaining the original train/validation splits. The new LMDB files should be named `train_ghosted.lmdb` and `val_ghosted.lmdb`.

## Identify original splits

### Subtask:
Determine which samples in `new_data_list` correspond to the original training and validation sets.


## Create new lmdb environments

### Subtask:
Create two new LMDB environments, one for the ghosted training data and one for the ghosted validation data.


**Reasoning**:
Create two LMDB environments for the ghosted training and validation data.



In [16]:
import lmdb
import os

# Define LMDB file paths
train_lmdb_path = 'train_ghosted.lmdb'
val_lmdb_path = 'val_ghosted.lmdb'

# Define a reasonable map_size (can be adjusted based on data size)
# 1e11 is 100 GB, adjust as needed
map_size = 1e10

# Create the training LMDB environment
train_env = lmdb.open(train_lmdb_path, map_size=map_size, create=True, subdir=False, readonly=False)

# Create the validation LMDB environment
val_env = lmdb.open(val_lmdb_path, map_size=map_size, create=True, subdir=False, readonly=False)

print(f"Created LMDB environment: {train_lmdb_path}")
print(f"Created LMDB environment: {val_lmdb_path}")

MemoryError: train_ghosted.lmdb: Cannot allocate memory